# Create and submits a mobotix job to the Waggle scheduler.

It is expected to monitors or trigger by an event (`scan_event`) to submit the jobs.

My idea is to,

0. A model or radar analyzer running in the HPC will know about the detected/expected weather system.
1. This code will then dynamically generates a YAML file for the specified direction for IR scanning with direction to scan are passed as a list (e.g., ["NW", "SW", "NE"]) for the my plugin arguments.
2. Then submits the job using `sesctl`.
    - Sets the SES environment variables (`SES_HOST` and `SES_USER_TOKEN`).
    - Creates the job and the `job_id` is extracted from the response.
    - The job is then submitted to the scheduler using the `job_id`.

3. The driving function or the main script will monitor the `scan_event` variables or ENV or may be a thread something like that. If it is `True`, it calls the job creation and submission functions.

## Setting Up `sesctl`

1. **Download `sesctl`**:
    - Visit the [Waggle Edge Scheduler Releases](https://github.com/waggle-sensor/edge-scheduler/releases/) page.
    - For Mac users, download the [sesctl-darwin-amd64](https://github.com/waggle-sensor/edge-scheduler/releases/download/0.27.2/sesctl-darwin-amd64) executable.
2.  `sesctl` is a stand alone executable program that runs from terminal.
3. **Set Up Environment Variables**:
    - Open your terminal and set the following environment variables:

      ```sh
      export SES_HOST=https://es.sagecontinuum.org
      export SES_USER_TOKEN=<<VALID TOKEN>>
      ```


In [27]:
import subprocess
import os

If we have multiple configurations created then we do not need this function. However, for the custom configuration for each events, like camera scan in perticular direction, we can modify the and  make it more flexible.

In [28]:
import yaml

def create_job_file(directions, nodes, ip, username, password, south, filename="dynamic_scan_job.yaml"):
    """
    Creates a job file dynamically for the Waggle scheduler.

    Parameters:
        directions (list): List of directions for scanning (e.g., ["NEH", "NEB", "NEG"]).
        nodes (dict): Dictionary of node names and their statuses (e.g., {"W020": True}).
        ip (str): IP address of the camera (e.g., "camera-mobotix-thermal").
        username (str): Username for the camera (e.g., "admin").
        password (str): Password for the camera (e.g., "wagglesage").
        south (str): South parameter value (e.g., "22").
        filename (str): The name of the output YAML file.

    Returns:
        str: The name of the generated job file.
    """
    job = {
        "name": "mobotix-scan-direction",
        "plugins": [
            {
                "name": "mobotix-scan-direction",
                "pluginSpec": {
                    "image": "registry.sagecontinuum.org/bhupendraraut/mobotix-scan:0.24.8.20",
                    "args": [
                        "--ip",
                        ip,
                        "--mode",
                        "direction",
                        "-south",
                        south,
                        "-pt",
                        f"{','.join(directions)}",
                        "-u",
                        username,
                        "-p",
                        password
                    ]
                }
            }
        ],
        "nodes": nodes,
        "scienceRules": [
            'schedule("mobotix-scan-direction"): cronjob("mobotix-scan-direction", "* * * * *")'
        ],
        "successCriteria": []
    }

    with open(filename, "w") as file:
        yaml.dump(job, file, default_flow_style=False)
    print(f"Job file {filename} created successfully.")
    return filename


In [29]:
directions = ["NEH", "NEB", "NEG", "EH", "EB", "EG", "SEH", "SEB", "SEG", "SH", "SB", "SG", "SWH", "SWB", "SWG"]
nodes = {"W021": True, "V032": True}
ip = "camera-mobotix-thermal"
username = "admin"
password = "meinsm"
south = "15"

# Create a job file
job_file = create_job_file(directions, nodes, ip, username, password, south)
print(f"Generated job file: {job_file}")


Job file dynamic_scan_job.yaml created successfully.
Generated job file: dynamic_scan_job.yaml


In [36]:
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def submit_job(filename):
    try:
        logging.info("Setting SES environment variables.")
        subprocess.run(["export", "SES_HOST=https://es.sagecontinuum.org"], shell=True)
        subprocess.run(["sh", "sesctl_env.sh"], shell=True)
        
        logging.info("Fetching SES_USER_TOKEN from environment.")
        token = os.environ.get('SES_USER_TOKEN')
        if not token:
            raise ValueError("API token not found in environment")
        else:
            logging.info("API token found.")

        logging.info("Creating job.")
        result = subprocess.run(["./sesctl-darwin-amd64", "create", "--file-path", filename], check=True, capture_output=True, text=True)
        logging.info(f"Job creation response: {result.stdout}")
        
        logging.info("Extracting job_id from the response.")
        job_id = yaml.safe_load(result.stdout).get("job_id")
        if not job_id:
            raise ValueError("Job ID not found in the response.")
        
        logging.info(f"Submitting job with job_id: {job_id}.")
        result = subprocess.run(["sesctl", "submit", "--job-id", job_id], check=True, capture_output=True, text=True)
        logging.info(f"Job submission response: {result.stdout}")
    except subprocess.CalledProcessError as e:
        logging.error(f"Error during job submission: {e.stderr}")
    except Exception as e:
        logging.error(f"An error occurred: {e}")


In [38]:
submit_job("dynamic_scan_job.yaml")

2024-12-19 10:42:00,744 - INFO - Setting SES environment variables.


export APPLICATIONINSIGHTS_CONFIGURATION_CONTENT="{}"
export APPLICATION_INSIGHTS_NO_DIAGNOSTIC_CHANNEL="1"
export CLICOLOR="1"
export CLICOLOR_FORCE="1"
export COMMAND_MODE="unix2003"
export CONDA_DEFAULT_ENV="data"
export CONDA_EXE="/Users/bhupendra/anaconda3/bin/conda"
export CONDA_PREFIX="/Users/bhupendra/anaconda3/envs/data"
export CONDA_PREFIX_1="/Users/bhupendra/anaconda3"
export CONDA_PREFIX_2="/Users/bhupendra/anaconda3/envs/data"
export CONDA_PREFIX_3="/Users/bhupendra/anaconda3"
export CONDA_PROMPT_MODIFIER="(data) "
export CONDA_PYTHON_EXE="/Users/bhupendra/anaconda3/bin/python"
export CONDA_ROOT="/Users/bhupendra/anaconda3"
export CONDA_SHLVL="4"
export CPL_ZIP_ENCODING="UTF-8"
export ELECTRON_RUN_AS_NODE="1"
export FORCE_COLOR="1"
export GDAL_DATA="/Users/bhupendra/anaconda3/envs/data/share/gdal"
export GDAL_DRIVER_PATH="/Users/bhupendra/anaconda3/envs/data/lib/gdalplugins"
export GIT_PAGER="cat"
export GSETTINGS_SCHEMA_DIR="/Users/bhupendra/anaconda3/envs/data/share/glib

KeyboardInterrupt: 

My environment was not set so this did not work, but it submitted the job before when the nv was set properly. I need to fix the part where e are setting up the environment.